In [1]:
import keras
from keras.datasets import mnist
import numpy as np
import gzip
import pickle
import struct

def load_mnist(path, kind='train'):
    import os
    import struct
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        struct.unpack('>II', lbpath.read(8))
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8)

    with gzip.open(images_path, 'rb') as imgpath:
        struct.unpack(">IIII", imgpath.read(16))
        images = np.frombuffer(imgpath.read(), dtype=np.uint8).reshape(len(labels), 784)

    return images, labels


Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)


In [3]:
x_train, y_train = load_mnist('fashion', kind='train')
x_test, y_test = load_mnist('fashion', kind='t10k')
x_train = np.expand_dims(x_train.reshape([-1, 28, 28]), -1)
x_test = np.expand_dims(x_test.reshape([-1, 28, 28]), -1)
y_train = keras.utils.np_utils.to_categorical(y_train)
y_test = keras.utils.np_utils.to_categorical(y_test)

In [4]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1]),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64, (5, 5), padding="same"),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(keras.optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, validation_split=0.10, batch_size=50, epochs=5)

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 150s - loss: 0.4027 - acc: 0.8587 - val_loss: 0.3129 - val_acc: 0.8818
Epoch 2/5
54000/54000 [==============================] - 149s - loss: 0.2658 - acc: 0.9039 - val_loss: 0.2537 - val_acc: 0.9060
Epoch 3/5
18000/54000 [=========>....................] - ETA: 96s - loss: 0.2151 - acc: 0.9206

In [ ]:
result = model.test_on_batch(x_test, y_test)

In [ ]:
print("test accuracy: {:.2f}".format(100*result[1]))